In [69]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import xgboost as xgb
#conda install -c jjhelmus py-xgboost-gpu
import os
import math

from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

# Though the following import is not directly being used, it is required
# for 3D projection to work
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn import datasets
import gensim

In [70]:
os.getcwd()
os.chdir('C:/Users/Hoc/OneDrive/My Documents/Machine Learning/kaggle/ames/')
#os.chdir('C:/Users/Richard/OneDrive/My Documents/Machine Learning/kaggle/ames/')

In [71]:
train = pd.read_csv('train.csv', na_values='NA')
#print(train.head())
test = pd.read_csv('test.csv', na_values='NA')
#print(test.head())

In [72]:
train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [82]:
print(train.OverallCond.dtypes)
print(train.OverallQual.dtypes)

int64
int64


In [84]:
train['OverallCond']= train.OverallCond.astype('object')
train['OverallQual']= train.OverallQual.astype('object')
print(train.OverallCond.dtypes)
print(train.OverallQual.dtypes)

object
object


In [178]:
text=pd.read_table('PropDes.txt',names=['Id2','Des'])

In [179]:
text.head()

Id2                                                Des
0  20  1-STORY 1946 & NEWER ALL STYLES dwelling invol...
1  30  1-STORY 1945 & OLDER dwelling involved in the ...
2  40  1-STORY W/FINISHED ATTIC ALL AGES dwelling inv...
3  45  1-1/2 STORY - UNFINISHED ALL AGES dwelling inv...
4  50  1-1/2 STORY FINISHED ALL AGES dwelling involve...

In [180]:
train.dtypes
text.dtypes

Id2    object
Des    object
dtype: object

In [181]:
text['Id2'] = text.Id2.str.strip()

In [182]:
text[text.Id2=='RL']

Id2                                                Des
21  RL  Residential Low Density zoning classification ...

In [183]:
pd.merge(train[['Id','MSZoning']], text,  left_on=train.MSZoning, right_on=text.Id2, how='left').head().iloc[:,-1]

0    Residential Low Density zoning classification ...
1    Residential Low Density zoning classification ...
2    Residential Low Density zoning classification ...
3    Residential Low Density zoning classification ...
4    Residential Low Density zoning classification ...
Name: Des, dtype: object

In [184]:
df = pd.DataFrame({"Des":''},index=train.Id)

In [185]:
categorical_features = train.select_dtypes(include = ["object"]).columns
numerical_features = train.select_dtypes(exclude = ["object"]).columns

In [186]:
for i, c in enumerate(categorical_features):
    df['Des'+str(i)] = pd.merge(train, text, left_on=train[c], right_on=text.Id2, how='left').iloc[:,-1]

In [187]:
for c in df.columns:
    df[c] = '. '+df[c]

In [188]:
df['Des']=df.fillna('').sum(axis=1)

In [189]:
df = pd.DataFrame(df.Des)

In [190]:
df.to_csv('des.csv', header=df.columns)

Using TensorFlow backend.


Index(['raw_text', 'group_1', 'group_2', 'group_3', 'group_4', 'group_5',
       'text_length', 'text_word_count', 'outcome'],
      dtype='object')

Index(['group_1', 'group_2', 'group_3', 'group_4', 'group_5', 'text_length',
       'text_word_count'],
      dtype='object')

0    12.247699
1    12.109016
2    12.317171
3    11.849405
4    12.429220
Name: outcome, dtype: float64